In [1]:
!pip install wandb
!pip install python-dotenv


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [3]:
!pwd

/teamspace/studios/this_studio/mlprojectcars/notebooks


In [2]:
cd mlprojectcars/notebooks/

/teamspace/studios/this_studio/mlprojectcars/notebooks


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

import os
import wandb
from dotenv import load_dotenv

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import make_scorer, mean_absolute_error, r2_score
from sklearn.decomposition import PCA

import joblib

from sklearn.impute import SimpleImputer

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import (
    MinMaxScaler,
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler,
    PolynomialFeatures,
    FunctionTransformer,
)

# Definamos el `random_state`:
random_state = 42

# Cambiemos la fuente de las gráficas de matplotlib:
plt.rc('font', family='serif', size=12)

In [5]:
# Load environment variables from .env file
load_dotenv('../api_keys.env')

# Get the API key from environment variable
api_key = os.getenv('WANDB_API_KEY')

# Log in to wandb using the API key
wandb.login(key=api_key)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jdmartinev (jdmartinev_). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /teamspace/studios/this_studio/.netrc


True

In [6]:
DATASET_LOC = '../data/'
prepared_data_file = 'train_df_transformed.csv'
df = pd.read_csv(os.path.join(DATASET_LOC,prepared_data_file))

In [7]:
num_cols = ['year','mileage','tax','milesPerGallon','engineSize']
cat_cols = ['brand_model','transmission','fuelType']
target = 'price'

In [8]:
X_train = df[cat_cols+num_cols]
y_train = df[target]

In [10]:
#Transformaciones para variables categoricas
categorical_transformer = Pipeline(steps=[
    ("encoder", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)),
    #('imputer', SimpleImputer(strategy='median')),
])

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    #('scaler', MinMaxScaler())
])

In [11]:
# Definimos el Pipeline de procesamiento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num_cols),  # Numerical columns with mean imputer
        ('cat', categorical_transformer, cat_cols)  # Categorical columns
    ],
    remainder='passthrough'  # This will passthrough any remaining columns
)

In [12]:
#Definimos nuestro regresor
rf_base = RandomForestRegressor(random_state=random_state)
gb_base = GradientBoostingRegressor(random_state=random_state)

In [15]:
# Definamos la malla de parámetros sobre la que haremos la búsqueda:
param_grid = {
    'regressor__n_estimators': [50, 100],
    'regressor__max_depth': list(range(6, 10)),
    'regressor__min_samples_leaf': [10, 300, 2000]
}

scorers = {
    'r2': make_scorer(r2_score),
    'mae': make_scorer(mean_absolute_error),
}

In [13]:
pipeline_rf = Pipeline(steps = [
    ('preprocessor', preprocessor),
    ('regressor', rf_base),
                            ])

pipeline_gb = Pipeline(steps = [
    ('preprocessor', preprocessor),
    ('regressor', gb_base),
    ])

In [16]:
# Definamos nuestros modelo mediante GridSearchCV:
rf = GridSearchCV(pipeline_rf, cv=3, param_grid=param_grid, scoring=scorers, refit='r2', n_jobs=-1, return_train_score=True)
gb = GridSearchCV(pipeline_gb, cv=3, param_grid=param_grid, scoring=scorers, refit='r2', n_jobs=-1, return_train_score=True)

In [17]:
%%time
# Entrenemos el random forest:
rf.fit(X_train, y_train)
# Entrenemos el gradient boosting:
gb.fit(X_train, y_train)

CPU times: user 7.26 s, sys: 177 ms, total: 7.44 s
Wall time: 1min 26s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer())]),
                                                                         ['year',
                                                                          'mileage',
                                                                          'tax',
                                                                          'milesPerGallon',
                                                                          'engineSize']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('encoder',
                                                                                          OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                                         unknown_value=-1))]),
                                                                         ['brand_model',
                                                                          'transmission',
                                                                          'fuelType'])])),
                                       ('regressor',
                                        GradientBoostingRegressor(random_state=42))]),
             n_jobs=-1,
             param_grid={'regressor__max_depth': [6, 7, 8, 9],
                         'regressor__min_samples_leaf': [10, 300, 2000],
                         'regressor__n_estimators': [50, 100]},
             refit='r2', return_train_score=True,
             scoring={'mae': make_scorer(mean_absolute_error),
                      'r2': make_scorer(r2_score)})

In [18]:
# start a new wandb run to track this script
config=rf.best_params_
config['method'] = 'Random Forest'
wandb.init(
    # set the wandb project where this run will be logged
    project="MLClase03",

    # track hyperparameters and run metadata
    config=config
)
best_index = rf.best_index_
best_results = rf.cv_results_
for metric in scorers.keys():
    wandb.log({
        f'best_{metric}_mean': best_results[f'mean_test_{metric}'][best_index],
        f'best_{metric}_std': best_results[f'std_test_{metric}'][best_index],
    })
wandb.finish()

best_mae_mean,▁
best_mae_std,▁
best_r2_mean,▁
best_r2_std,▁
best_mae_mean,1800.86366
best_mae_std,706.34247
best_r2_mean,0.83627
best_r2_std,0.06427


In [19]:
# start a new wandb run to track this script
config=gb.best_params_
config['method'] = 'Gradient boosting'
wandb.init(
    # set the wandb project where this run will be logged
    project="MLClase03",
    name = 'Gradient Boosting'
    # track hyperparameters and run metadata
    config=config
)
best_index = gb.best_index_
best_results = gb.cv_results_
for metric in scorers.keys():
    wandb.log({
        f'best_{metric}_mean': best_results[f'mean_test_{metric}'][best_index],
        f'best_{metric}_std': best_results[f'std_test_{metric}'][best_index],
    })
    print(best_results[f'mean_test_{metric}'][best_index])
wandb.finish()

0.856139930586347
1681.9912358181857


best_mae_mean,▁
best_mae_std,▁
best_r2_mean,▁
best_r2_std,▁
best_mae_mean,1681.99124
best_mae_std,908.837
best_r2_mean,0.85614
best_r2_std,0.08383


In [20]:
# Create a new dictionary without the 'regressor__' prefix
cleaned_params = {key.replace('regressor__', ''): value for key, value in gb.best_params_.items()}
cleaned_params.pop("method")
X_train_prep = preprocessor.fit_transform(X_train)
gb_model = GradientBoostingRegressor(**cleaned_params)
gb_model.fit(X_train_prep,y_train)


GradientBoostingRegressor(max_depth=9, min_samples_leaf=10, n_estimators=50)

In [38]:
#Save the preprocessor and the classifier
# Save the preprocessor
joblib.dump({'cols':num_cols+cat_cols, 'preprocessor':preprocessor}, os.path.join(DATASET_LOC,'model','preprocessor.pkl'))
# Save the model
joblib.dump(gb_model, os.path.join(DATASET_LOC,'model','model.pkl'))

['../data/model/model.pkl']

In [27]:
print("Model: Gradient boosting")
print(f'R^2: {gb_model.score(X_train_prep, y_train)}')
print(f'MAE: {mean_absolute_error(y_train, gb_model.predict(X_train_prep))}')
print('\n')

Model: Gradient boosting
R^2: 0.9696408371711306
MAE: 813.2023709266927




In [29]:
test_data_file = 'test_df_transformed.csv'
test_df = pd.read_csv(os.path.join(DATASET_LOC,test_data_file))

In [30]:
X_test = test_df[cat_cols+num_cols]
y_test = test_df[target]

In [31]:
X_test_prep = preprocessor.transform(X_test)

In [32]:
print("Model: Gradient boosting")
print(f'R^2: {gb_model.score(X_test_prep, y_test)}')
print(f'MAE: {mean_absolute_error(y_test, gb_model.predict(X_test_prep))}')
print('\n')

Model: Gradient boosting
R^2: 0.9696408371711306
MAE: 813.2023709266927




In [33]:
# start a new wandb run to track this script
config=gb.best_params_
config['method'] = 'Gradient boosting'
wandb.init(
    # set the wandb project where this run will be logged
    project="cars_gridsearch_cv_project",
    # track hyperparameters and run metadata
    config=config
)
best_index = gb.best_index_
best_results = gb.cv_results_
for metric in scorers.keys():
    wandb.log({
        'r2_test': gb_model.score(X_test_prep, y_test),
        'mae_test': mean_absolute_error(y_test, gb_model.predict(X_test_prep)),
    })
wandb.finish()

mae_test,▁▁
r2_test,▁▁
mae_test,813.20237
r2_test,0.96964
